<a href="https://colab.research.google.com/github/promaprogga/iResSENet-An-Accurate-Convolutional-Neural-Network-for-Retinal-Blood-Vessel-Segmentation/blob/main/model/iResSENet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from keras.layers import *
from keras.backend import int_shape
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Add, BatchNormalization,  Activation, MaxPool2D 
from tensorflow.keras.layers import Conv2DTranspose, Concatenate, Input, GlobalAveragePooling2D, Reshape, Dense, multiply

# Convolutional block
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("gelu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("gelu")(x)

    return x

# Squeeze and excitation block  
def se_block(block_input, num_filters, ratio=8):                            

	pool1 = GlobalAveragePooling2D()(block_input)
	flat = Reshape((1, 1, num_filters))(pool1)
	dense1 = Dense(num_filters//ratio, activation='gelu')(flat)
	dense2 = Dense(num_filters, activation='sigmoid')(dense1)
	scale = multiply([block_input, dense2])
	
	return scale


# Single ResNet block
def resnet_block(block_input, num_filters):                                  


	if int_shape(block_input)[3] != num_filters:
		block_input = Conv2D(num_filters, kernel_size=(1, 1))(block_input)
	
	conv1 = Conv2D(num_filters, kernel_size=(1, 1), padding='same')(block_input)
	norm1 = BatchNormalization()(conv1)
	relu1 = Activation('gelu')(norm1)
	conv2 = Conv2D(num_filters, kernel_size=(3,3), padding='same')(relu1)
	norm2 = BatchNormalization()(conv2)
	
	se = se_block(norm2, num_filters=num_filters)
	
	sum = Add()([block_input, se])
	relu2 = Activation('gelu')(sum)
	
	return relu2



# Single Encoder block
def encoder_block(inputs, num_filters, length):
    x = resnet_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
  
    return x, p


# Single Decoder block
def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

# iResSENet
def build_iResSENet(input_shape):
    inputs = Input(input_shape)

    p0 = inputs
    s1, p1 = encoder_block(p0, 32, 4)
    s2, p2 = encoder_block(p1, 64, 3)
    s3, p3 = encoder_block(p2, 128, 2)
    s4, p4 = encoder_block(p3, 256, 1)

    b1 = conv_block(p4, 512)

    d1 = decoder_block(b1, s4, 256)
    d2 = decoder_block(d1, s3, 128)
    d3 = decoder_block(d2, s2, 64)
    d4 = decoder_block(d3, s1, 32)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="iResSENet")
    return model


input_shape = (512, 512, 3)
model = build_iResSENet(input_shape)
model.summary()

Model: "iResSENet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_95 (Conv2D)             (None, 512, 512, 32  128         ['input_6[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_96 (Conv2D)             (None, 512, 512, 32  1056        ['conv2d_95[0][0]']              
                                )                                                         